In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# initialize an instance of the chrome driver (browser)
driver = webdriver.Chrome()
# visit your target site
driver.get('https://callink.berkeley.edu/Organizations')

In [8]:
# Load all pages
while driver.find_elements(By.TAG_NAME, 'button'):
    load_button = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.TAG_NAME, 'button')))
    load_button.location_once_scrolled_into_view
    load_button.click()

In [9]:
# Grab all links
a_tags = driver.find_elements(By.TAG_NAME, 'a')
links = []
for a in a_tags:
    link = a.get_attribute('href')
    if link.startswith('https://callink.berkeley.edu/organization/'):
        links.append(link)

In [10]:
# 'description' is currently html code
fields = ['name', 'shortName', 'email', 'description', 'summary', 'status', 'socialMedia', 'primaryContact', 'profilePicture', 'organizationType']

In [11]:
def scrape(URL):
    data = {}
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    extract = soup.findAll('script')[8]
    text = extract.string[25:-1]

    dictionary = json.loads(text)
    orgData = dictionary['preFetchedData']['organization']
    for field in fields:
        data[field] = orgData[field]
    return data
    

In [12]:
json_data = []
for link in links:
    json_data.append(scrape(link))

In [13]:
file_path = "nectar.json"

# Write the list to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(json_data, json_file)